#  Probability map  

 
1. Load model saved in disc
2. Regularly sample a input WSI and execute image preprocessing. The position of each individual tile is tracked
3. Predict the class of tach tile,obtain probablities
4. Reassemble the tiles. Only tissue tiles are given with probalities and the probablities of non-tissue tiles are zero. 
5. Build heatmap

In [246]:
import tensorflow as tf
import cv2
import math
import os 
import numpy as np
import pandas as pd

def ImgSamplingColorSpaceTransform(parent_path,image_filename,output_path):
    NonTissue_Position_collection = []
    #img_record = pd.DataFrame ( columns = ['TissueTile_Position','TissueTile_Predition'])
    TissueTile_Position_collection = []
    TissueTile_filename_collection = []
    #TissueTile_Predition_Prob_collection = []
    all_cropped_images = []
    #Dataframe
    tile_size = (101, 101)
    offset = (101, 101) 
    path_WSI = os.path.join(parent_path,image_filename)
    img_WSI = cv2.imread(path_WSI) #array
    img_shape = img_WSI.shape
    height  = int(math.ceil(img_shape[0]/(offset[1] * 1.0)))
    width = int(math.ceil(img_shape[1]/(offset[0] * 1.0)))
    num = 0
    for i in range(height): #height
        for j in range( width ):#width
            #for each tile
            #step1. tile sampling
            cropped_img_WSI = img_WSI[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
            all_cropped_images.append(cropped_img_WSI)
            if cropped_img_WSI is not None:
                cropped_image_WSI_01 = cropped_img_WSI/255 
                cropped_image_WSI_img_01_mean = cropped_image_WSI_01.mean()
                cropped_image_WSI_img_01_std = cropped_image_WSI_01.std() 
                #step2.determine non_tissue
                if cropped_image_WSI_img_01_mean >= 0.85  and cropped_image_WSI_img_01_std <= 0.1: #more strict 
                        NonTissue_Position_collection.append((i,j) )
                else:#tissue region,  
                        #step3.  preprocessing,standization, normalized to [0,1]k
                        img_processed  = cv2.cvtColor(cropped_img_WSI,cv2.COLOR_RGB2YUV)
                        TissueTile_Position_collection.append((i,j))
                        TissueTile_filename_collection.append(image_filename[0:-4]+'_tile_'+ str(i)+"_"+ str(j)+'.png')
                        cv2.imwrite(output_path + '\\'+ image_filename[0:-4]+'_tile_'+ str(i)+"_"+ str(j)+'.png',img_processed)        
            
    return  all_cropped_images,NonTissue_Position_collection,TissueTile_Position_collection, TissueTile_filename_collection,height,width


In [328]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def Display_ProbablityMap(model_path,parent_path,image_filename,output_path): 
    if  os.path.exists(output_path) :
        shutil.rmtree(output_path)     
    os.makedirs(output_path)
    all_cropped_images,NonTissue_Position_collection,TissueTile_Position_collection, TissueTile_filename_collection, height,width = ImgSamplingColorSpaceTransform(
    parent_path,image_filename,output_path) 
    model_path = r'\ECSE484_Fall2020\tensorflow_model\mymodel_3conv_2FC_dataset2'
    mymodel = tf.keras.models.load_model(model_path)
    ProbabilityMap_TissueTiles_path = r'\ECSE484_Fall2020\ProbabilityMap_TissueTiles'

    test_datagen = ImageDataGenerator(rescale=1./255,
      samplewise_std_normalization = True,
      samplewise_center = True)
    test_generator = test_datagen.flow_from_directory(
        ProbabilityMap_TissueTiles_path ,  # this is the target directory
        target_size=(101,101), 
        classes=['test'],
        shuffle = False) #the order of TissueTile_filename_collection, TissueTile_Position_collection and invasive_pro_collection are the same
        #batch_size=20 # since we use binary_crossentropy loss, we need binary labels
    predict_results = mymodel.predict(test_generator)
    invasive_pro_collection = predict_results[:,1]
    prob_mat = np.zeros([height ,width])
# idx/width: height ,  idx%width:width
#returned : all_cropped_images,NonTissue_Position_collection,TissueTile_Position_collection,
#TissueTile_filename_collection, height,width
    for idx, img in enumerate(all_cropped_images):
        if (int(idx/width), int(idx%width)) in NonTissue_Position_collection:
            prob_mat[ int(idx/width),int(idx%width)] = 0
        if (int(idx/width), int(idx%width)) in TissueTile_Position_collection:
            tile_index = TissueTile_Position_collection.index(  (int(idx/width), int(idx%width))   )
            prob_mat[int(idx/width),int(idx%width)]  = invasive_pro_collection[tile_index]
    fig,ax  = plt.subplots(figsize=(200,150))
    im = plt.pcolor(prob_mat, cmap= plt.cm.Reds , vmin= 0, vmax= 1 )
    fig.colorbar(im,ax = ax )  

    plt.savefig(os.path.join('\ECSE484_Fall2020', 'heatmap_1.png'))
    plt.show()
     

In [329]:
import os
import random
import shutil

model_path = r'\ECSE484_Fall2020\tensorflow_model\mymodel_3conv_2FC_dataset2'
parent_path = r'\ECSE484_Fall2020\testing_2020\CINJ_imgs_idx5'
image_file_list =  os.listdir(parent_path)
temp = random.randrange(0, len(image_file_list ))
image_filename = image_file_list[temp]
output_path = r'\ECSE484_Fall2020\ProbabilityMap_TissueTiles\test'
Display_ProbablityMap(model_path,parent_path,image_filename,output_path)  

Found 337 images belonging to 1 classes.


In [331]:
image_filename

'10303.png'